In [1]:
%pylab 
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation
from landlab.components.conduit_networks.calcite import calcite_diss_palmer_transfer

from PIL import Image
from PIL import ImageDraw

def plot_links(grid, value_name, autoscale=True, 
               vmin=0., vmax=0., cmap_name='viridis', 
               magnitude= False, lw=5, 
               pixels_per_node=31, x_name='X', 
               y_name='Y', var_name='', use_PIL=True):#, logvalues=False ):
    if use_PIL:
        pixel_xscaling_factor = pixels_per_node/grid.dx
        pixel_yscaling_factor = pixels_per_node/grid.dy
        im = Image.new('RGBA', ( int(round(grid.number_of_cell_columns*pixels_per_node)), int(round(grid.number_of_cell_rows*pixels_per_node)) ), (255,255,255,255))
        draw = ImageDraw.Draw(im)

    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]       
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    #if logvalues:
    #    values = np.log10(abs(grid.at_link[value_name]))
    
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    if use_PIL:
        for i, value in enumerate(values):
            draw.line( ((link_head_x[i]*pixel_xscaling_factor,link_head_y[i]*pixel_yscaling_factor),(link_tail_x[i]*pixel_xscaling_factor,link_tail_y[i]*pixel_yscaling_factor)),fill=scalarMap.to_rgba(value,bytes=True),width=lw) 
        imshow(np.asarray(im), origin='lower', extent=(0,grid.number_of_cell_columns,0,grid.number_of_cell_rows))
    else:
        for i, value in enumerate(values):
            xs = [link_head_x[i],link_tail_x[i]]
            ys = [link_head_y[i],link_tail_y[i]]
            img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))    
    cb = colorbar(scalarMap)
    cb.ax.set_ylabel(var_name)
    xlabel(x_name)
    ylabel(y_name)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
mg = RasterModelGrid((5,5),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
h = mg.add_zeros('node', 'hydraulic__head')
Q = mg.add_zeros('link', 'conduit__discharge')
mg.add_zeros('node','storage')
mg.at_node['storage']=10.*np.ones(mg.number_of_nodes)


w = mg.add_zeros('link','width')
mg.at_link['width'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'] = w


#Set up link elevation offsets
xc_min_head = mg.add_zeros('link', 'conduit_head__offset')
xc_min_tail = mg.add_zeros('link', 'conduit_tail__offset')
y_max = mg.add_zeros('link', 'maximum__depth')
mg.at_link['maximum__depth']= mg.at_link['hydraulic__diameter']#applicable for square conduits



#set heads at edges
h[mg.nodes_at_left_edge] = 10.
h[mg.nodes_at_right_edge] = 0.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.

mg.set_closed_boundaries_at_grid_edges(False,True,False,True)
Q[mg.active_links] = 0.1#0.1*np.random.rand(mg.number_of_active_links)
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print("Number of links = ", mg.number_of_links)
print("Number of nodes = ", mg.number_of_nodes)
print("Number of active links = ", mg.number_of_active_links)
print("Number of core nodes = ", mg.number_of_core_nodes)


pfn = PresFlowNetwork(mg)


Number of links =  40
Number of nodes =  25
Number of active links =  18
Number of core nodes =  9


In [5]:
for i in arange(500):
    print('Time iteration:', i)
    pfn.dyn_wave_solution(dt=5000.)

Time iteration: 0
max h_old= 10.0
max Q_old = 7.05984451036e-06
max h_old= 10.0
max Q_old = 7.05984451036e-06
max change in h:  0.00100074913973
max h_old= 10.0
max Q_old = 7.05984451036e-06
max change in h:  0.000500691297521
max h_old= 10.0
max Q_old = 7.05984451036e-06
max change in h:  0.000250521099445
max h_old= 10.0
max Q_old = 7.05984451036e-06
max change in h:  0.000125356455321
max h_old= 10.0
max Q_old = 7.05984451036e-06
max change in h:  6.27302063996e-05
Time iteration: 1
max h_old= 10.0
max Q_old = 7.06135893557e-06
max h_old= 10.0
max Q_old = 7.06135893557e-06
max change in h:  0.00100211627307
max h_old= 10.0
max Q_old = 7.06135893557e-06
max change in h:  0.000501366903248
max h_old= 10.0
max Q_old = 7.06135893557e-06
max change in h:  0.000257282960619
max h_old= 10.0
max Q_old = 7.06135893557e-06
max change in h:  0.000131742117819
max h_old= 10.0
max Q_old = 7.06135893557e-06
max change in h:  6.30019869274e-05
Time iteration: 2
max h_old= 10.0
max Q_old = 7.062880

max h_old= 10.0
max Q_old = 7.11089128078e-06
max change in h:  0.00104005402316
max h_old= 10.0
max Q_old = 7.11089128078e-06
max change in h:  0.000520325598958
max h_old= 10.0
max Q_old = 7.11089128078e-06
max change in h:  0.000263021067026
max h_old= 10.0
max Q_old = 7.11089128078e-06
max change in h:  0.00013418941114
max h_old= 10.0
max Q_old = 7.11089128078e-06
max change in h:  6.51107401008e-05
Time iteration: 34
max h_old= 10.0
max Q_old = 7.11246586662e-06
max h_old= 10.0
max Q_old = 7.11246586662e-06
max change in h:  0.00104106704567
max h_old= 10.0
max Q_old = 7.11246586662e-06
max change in h:  0.000520830300883
max h_old= 10.0
max Q_old = 7.11246586662e-06
max change in h:  0.000260582684246
max h_old= 10.0
max Q_old = 7.11246586662e-06
max change in h:  0.000130383902559
max h_old= 10.0
max Q_old = 7.11246586662e-06
max change in h:  6.52424593093e-05
Time iteration: 35
max h_old= 10.0
max Q_old = 7.11404197996e-06
max h_old= 10.0
max Q_old = 7.11404197996e-06
max cha

max h_old= 10.0
max Q_old = 7.16687405115e-06
max change in h:  0.00108157875649
max h_old= 10.0
max Q_old = 7.16687405115e-06
max change in h:  0.000541099346573
max h_old= 10.0
max Q_old = 7.16687405115e-06
max change in h:  0.000270714013042
max h_old= 10.0
max Q_old = 7.16687405115e-06
max change in h:  0.000135443619403
max h_old= 10.0
max Q_old = 7.16687405115e-06
max change in h:  6.77672870184e-05
Time iteration: 69
max h_old= 10.0
max Q_old = 7.16849893368e-06
max h_old= 10.0
max Q_old = 7.16849893368e-06
max change in h:  0.00108286300634
max h_old= 10.0
max Q_old = 7.16849893368e-06
max change in h:  0.000541739675804
max h_old= 10.0
max Q_old = 7.16849893368e-06
max change in h:  0.000271004450948
max h_old= 10.0
max Q_old = 7.16849893368e-06
max change in h:  0.000135558436552
max h_old= 10.0
max Q_old = 7.16849893368e-06
max change in h:  6.78053196301e-05
Time iteration: 70
max h_old= 10.0
max Q_old = 7.17012517471e-06
max h_old= 10.0
max Q_old = 7.17012517471e-06
max ch

max change in h:  0.000562295419076
max h_old= 10.0
max Q_old = 7.22286110713e-06
max change in h:  0.000281311434847
max h_old= 10.0
max Q_old = 7.22286110713e-06
max change in h:  0.000140738579409
max h_old= 10.0
max Q_old = 7.22286110713e-06
max change in h:  7.04112198013e-05
Time iteration: 103
max h_old= 10.0
max Q_old = 7.22453012319e-06
max h_old= 10.0
max Q_old = 7.22453012319e-06
max change in h:  0.00112523237904
max h_old= 10.0
max Q_old = 7.22453012319e-06
max change in h:  0.000562940925224
max h_old= 10.0
max Q_old = 7.22453012319e-06
max change in h:  0.000281629169281
max h_old= 10.0
max Q_old = 7.22453012319e-06
max change in h:  0.000140891333953
max h_old= 10.0
max Q_old = 7.22453012319e-06
max change in h:  7.04835215748e-05
Time iteration: 104
max h_old= 10.0
max Q_old = 7.22620037244e-06
max h_old= 10.0
max Q_old = 7.22620037244e-06
max change in h:  0.00112649062635
max h_old= 10.0
max Q_old = 7.22620037244e-06
max change in h:  0.000563568572874
max h_old= 10.

max change in h:  0.000583368438293
max h_old= 10.0
max Q_old = 7.28180267261e-06
max change in h:  0.000291846619795
max h_old= 10.0
max Q_old = 7.28180267261e-06
max change in h:  0.000146002394151
max h_old= 10.0
max Q_old = 7.28180267261e-06
max change in h:  7.3039678302e-05
Time iteration: 136
max h_old= 10.0
max Q_old = 7.28028414484e-06
max h_old= 10.0
max Q_old = 7.28028414484e-06
max change in h:  0.00116735687531
max h_old= 10.0
max Q_old = 7.28028414484e-06
max change in h:  0.000584004275367
max h_old= 10.0
max Q_old = 7.28028414484e-06
max change in h:  0.000292166493394
max h_old= 10.0
max Q_old = 7.28028414484e-06
max change in h:  0.000146166139375
max h_old= 10.0
max Q_old = 7.28028414484e-06
max change in h:  7.31243745857e-05
Time iteration: 137
max h_old= 10.0
max Q_old = 7.28943248143e-06
max h_old= 10.0
max Q_old = 7.28943248143e-06
max change in h:  0.00116864507503
max h_old= 10.0
max Q_old = 7.28943248143e-06
max change in h:  0.000584645236782
max h_old= 10.0

...

max h_old= 10.0
max Q_old = 8.83362870219e-06
max change in h:  0.000717339624126
max h_old= 10.0
max Q_old = 8.83362870219e-06
max change in h:  0.000358842750035
max h_old= 10.0
max Q_old = 8.83362870219e-06
max change in h:  0.000179507916934
max h_old= 10.0
max Q_old = 8.83362870219e-06
max change in h:  8.9797265951e-05
Time iteration: 339
max h_old= 10.0
max Q_old = 8.84059823327e-06
max h_old= 10.0
max Q_old = 8.84059823327e-06
max change in h:  0.00143534986953
max h_old= 10.0
max Q_old = 8.84059823327e-06
max change in h:  0.000718020612924
max h_old= 10.0
max Q_old = 8.84059823327e-06
max change in h:  0.000359183291366
max h_old= 10.0
max Q_old = 8.84059823327e-06
max change in h:  0.000179678210964
max h_old= 10.0
max Q_old = 8.84059823327e-06
max change in h:  8.98824245947e-05
Time iteration: 340
max h_old= 10.0
max Q_old = 8.85077403964e-06
max h_old= 10.0
max Q_old = 8.85077403964e-06
max change in h:  0.00143671202829
max h_old= 10.0
max Q_old = 8.85077403964e-06
max c

max change in h:  9.26196250548e-05
Time iteration: 372
max h_old= 10.0
max Q_old = 9.1269429814e-06
max h_old= 10.0
max Q_old = 9.1269429814e-06
max change in h:  0.00148049519275
max h_old= 10.0
max Q_old = 9.1269429814e-06
max change in h:  0.000740596357076
max h_old= 10.0
max Q_old = 9.1269429814e-06
max change in h:  0.000370472698311
max h_old= 10.0
max Q_old = 9.1269429814e-06
max change in h:  0.00018532367868
max h_old= 10.0
max Q_old = 9.1269429814e-06
max change in h:  9.27055389743e-05
Time iteration: 373
max h_old= 10.0
max Q_old = 9.13354006149e-06
max h_old= 10.0
max Q_old = 9.13354006149e-06
max change in h:  0.00148186944273
max h_old= 10.0
max Q_old = 9.13354006149e-06
max change in h:  0.000741283574507
max h_old= 10.0
max Q_old = 9.13354006149e-06
max change in h:  0.000370816353063
max h_old= 10.0
max Q_old = 9.13354006149e-06
max change in h:  0.000185495528985
max h_old= 10.0
max Q_old = 9.13354006149e-06
max change in h:  9.27914755451e-05
Time iteration: 374
m

max change in h:  0.000764062679951
max h_old= 10.0
max Q_old = 9.42719339549e-06
max change in h:  0.000382207406703
max h_old= 10.0
max Q_old = 9.42719339549e-06
max change in h:  0.000191191803461
max h_old= 10.0
max Q_old = 9.42719339549e-06
max change in h:  9.56399852061e-05
Time iteration: 407
max h_old= 10.0
max Q_old = 9.42219458139e-06
max h_old= 10.0
max Q_old = 9.42219458139e-06
max change in h:  0.00152880806351
max h_old= 10.0
max Q_old = 9.42219458139e-06
max change in h:  0.000764755988085
max h_old= 10.0
max Q_old = 9.42219458139e-06
max change in h:  0.000382554105665
max h_old= 10.0
max Q_old = 9.42219458139e-06
max change in h:  0.00019136517531
max h_old= 10.0
max Q_old = 9.42219458139e-06
max change in h:  9.57266822734e-05
Time iteration: 408
max h_old= 10.0
max Q_old = 9.44569227915e-06
max h_old= 10.0
max Q_old = 9.44569227915e-06
max change in h:  0.00153019485321
max h_old= 10.0
max Q_old = 9.44569227915e-06
max change in h:  0.000765449472973
max h_old= 10.0

max h_old= 10.0
max Q_old = 9.51526763115e-06
max change in h:  0.00157615539854
max h_old= 10.0
max Q_old = 9.51526763115e-06
max change in h:  0.000788432674215
max h_old= 10.0
max Q_old = 9.51526763115e-06
max change in h:  0.000394393952885
max h_old= 10.0
max Q_old = 9.51526763115e-06
max change in h:  0.000197285848428
max h_old= 10.0
max Q_old = 9.51526763115e-06
max change in h:  9.8687392299e-05
Time iteration: 442
max h_old= 10.0
max Q_old = 9.97879596929e-06
max h_old= 10.0
max Q_old = 9.97879596929e-06
max change in h:  0.00157755403686
max h_old= 10.0
max Q_old = 9.97879596929e-06
max change in h:  0.000789132080792
max h_old= 10.0
max Q_old = 9.97879596929e-06
max change in h:  0.000394743699742
max h_old= 10.0
max Q_old = 9.97879596929e-06
max change in h:  0.00019746074357
max h_old= 10.0
max Q_old = 9.97879596929e-06
max change in h:  9.87748506889e-05
Time iteration: 443
max h_old= 10.0
max Q_old = 9.47783515319e-06
max h_old= 10.0
max Q_old = 9.47783515319e-06
max ch

max change in h:  0.000405677089063
max h_old= 10.0
max Q_old = 9.95681086292e-06
max change in h:  0.000202928104675
max h_old= 10.0
max Q_old = 9.95681086292e-06
max change in h:  0.000101508863398
max h_old= 10.0
max Q_old = 9.95681086292e-06
max change in h:  5.07768527411e-05
Time iteration: 474
max h_old= 10.0
max Q_old = 1.00542580365e-05
max h_old= 10.0
max Q_old = 1.00542580365e-05
max change in h:  0.0016226976154
max h_old= 10.0
max Q_old = 1.00542580365e-05
max change in h:  0.000811706638296
max h_old= 10.0
max Q_old = 1.00542580365e-05
max change in h:  0.000406032358219
max h_old= 10.0
max Q_old = 1.00542580365e-05
max change in h:  0.000203105760482
max h_old= 10.0
max Q_old = 1.00542580365e-05
max change in h:  0.000101597701884
max h_old= 10.0
max Q_old = 1.00542580365e-05
max change in h:  5.08212772585e-05
Time iteration: 475
max h_old= 10.0
max Q_old = 9.98897171497e-06
max h_old= 10.0
max Q_old = 9.98897171497e-06
max change in h:  0.00162411868889
max h_old= 10.0

In [6]:
Q

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         8.02391232e-06,  -6.58391990e-06,  -2.18205115e-06,
         6.40111863e-06,   0.00000000e+00,  -5.01429640e-06,
         4.27756958e-06,   4.68905774e-06,   0.00000000e+00,
         1.02438830e-05,   3.04579428e-07,   1.14294672e-06,
        -1.18283406e-06,   0.00000000e+00,   4.92424190e-06,
         2.91001401e-06,   4.26175063e-06,   0.00000000e+00,
         7.96794571e-06,  -3.96070802e-06,   3.31672492e-06,
        -4.42285145e-06,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00])

In [15]:
FUDGE=0.0001
active_links = mg.active_links
head_nodes = mg.node_at_link_head[active_links]
tail_nodes = mg.node_at_link_tail[active_links]
h_head = mg.at_node['hydraulic__head'][head_nodes]
h_tail = mg.at_node['hydraulic__head'][tail_nodes]
#Calculate flow depths using offset and junction elevations
y_head = h_head - mg.at_node['junction__elevation'][head_nodes] \
         - mg.at_link['conduit_head__offset'][active_links]
y_tail = h_tail - mg.at_node['junction__elevation'][tail_nodes] \
          - mg.at_link['conduit_tail__offset'][active_links]
y_head[y_head>mg.at_link['maximum__depth'][active_links]] = mg.at_link['maximum__depth'][active_links][y_head>mg.at_link['maximum__depth'][active_links]]
y_tail[y_tail>mg.at_link['maximum__depth'][active_links]] = mg.at_link['maximum__depth'][active_links][y_tail>mg.at_link['maximum__depth'][active_links]]
y_avg = 0.5*(y_head + y_tail)
A_avg = mg.at_link['width'][active_links] * y_avg
y_avg[y_avg<FUDGE] = FUDGE
A_avg[A_avg<FUDGE] = FUDGE

In [22]:
#y_tail[y_tail>mg.at_link['maximum__depth'][active_links]] = 
#mg.at_link['maximum__depth'][active_links][y_tail>mg.at_link['maximum__depth'][active_links]]
d_h_square(mg.at_link['width'][active_links], y_avg)

array([  3.96936126e-01,   3.53636005e-04,   3.99594057e-04,
         3.99687423e-04,   3.99814040e-04,   3.99715864e-04,
         3.99694894e-04,   4.90132384e-01,   3.99793756e-04,
         3.96927434e-04,   3.99777355e-04,   3.99802869e-04,
         3.99787176e-04,   3.97200621e-04,   2.81319838e-01,
         3.99646193e-04,   3.99446218e-04,   3.99054260e-04])

In [21]:
def d_h_square(width, flow_depth):
    d_H = np.zeros(np.size(width))
    is_full_pipe = np.isclose(width,flow_depth)
    d_H[is_full_pipe] = width[is_full_pipe]
    d_H[~is_full_pipe] = 4.*width[~is_full_pipe]*flow_depth[~is_full_pipe] / (2.*flow_depth[~is_full_pipe] + width[~is_full_pipe])
    return d_H

In [39]:
mg.at_link['maximum__depth'][active_links]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [5]:
d_h.copy()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])